# Preparation of the analysis
## Importing the libraries

In [ ]:
# ---------- Handling datasets ---------- #
import os
import gzip
import tarfile
import pandas as pd

# ---------- Compute calculations ---------- #
import datetime
import math
import numpy as np
import scipy.stats as stats

# ---------- Plots/figures/tables generation libraries ---------- #
import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette("Set1")
import matplotlib.pyplot as plt
import plotly as py
import plotly.tools as tls
import plotly.express as px 
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# ---------- Statistical and ML libraries ---------- #
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from textblob import TextBlob

## Importing the dataset

We first define the path to the dataset.

In [ ]:
PATH_BA = '../Data/BeerAdvocate_CSV.tar.gz'
PATH_RB = '../Data/RateBeer_CSV.tar.gz'
folder_BA = tarfile.open(PATH_BA)
folder_RB = tarfile.open(PATH_RB)

We now import the dataset and convert the different .csv files to pandas dataframes. This is done on both BeerAdvocate and RateBeer datasets. For each dataset we obtain four dataframes: one for the reviews, one for the beers, one for the breweries and one for the users.

In [ ]:
# Extracting the files for BeerAdvocate
df_beers_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/beers.csv')))
df_users_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/users.csv')))
df_reviews_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/reviews.csv')))
df_breweries_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/breweries.csv')))

# Extracting the files for RateBeer
df_beers_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/beers.csv')))
df_users_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/users.csv')))
df_reviews_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/reviews.csv')))
df_breweries_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/breweries.csv')))

## Preprocessing the dataframes

### Adding location information to the dataframes

For the review dataframes of both datasets, we want to add two columns that corresponds to the country of the user and the country of the brewery. We do this by extracting the information from the user and brewery dataframes and merging them with the review dataframe. In the end, we obtain `df_RB` and `df_BA` which have the reviews of RateBeer and BeerAdvocate respectively.

In [ ]:
#RateBeer
user_nat_RB=pd.DataFrame()
user_nat_RB['user_name']=df_users_RB['user_name']
user_nat_RB['location_user']=df_users_RB['location']
new_reviews_RB=pd.merge(df_reviews_RB, user_nat_RB,  how='inner', on='user_name')

beers_nat_RB=pd.DataFrame()
beers_nat_RB['beer_id']=df_breweries_RB['id']
beers_nat_RB['beers_location']=df_breweries_RB['location']
new_reviews_RB['beer_id']=new_reviews_RB['beer_id'].apply(lambda x: int(x))
df_RB= pd.merge(new_reviews_RB, beers_nat_RB, how='inner', on='beer_id')

#BeerAdvocate
user_nat_BA=pd.DataFrame()
user_nat_BA['user_name']=df_users_BA['user_name']
user_nat_BA['location_user']=df_users_BA['location']
new_reviews_BA=pd.merge(df_reviews_BA, user_nat_BA,  how='inner', on='user_name')

beers_nat_BA=pd.DataFrame()
beers_nat_BA['beer_id']=df_breweries_BA['id']
beers_nat_BA['beers_location']=df_breweries_BA['location']
new_reviews_BA['beer_id']=new_reviews_BA['beer_id'].apply(lambda x: int(x))
df_BA = pd.merge(new_reviews_BA, beers_nat_BA, how='inner', on='beer_id')

In [ ]:
df_BA

#### Adding states for the US based locations

The geographical information contain the name of the country for users and breweries. However, for the US based locations, we also have the name of the state. We want to extract this information and add it to the dataframe. To do so we create will create an extra column for both breweries and users that will contain the US postal abbreviations for each state.

We first extract the postal abbreviations for each state from wikipedia using `pd.read_html`. 

We then process the dataframe into one (`US_states`) containing a column for the postal abbreviations and another one for the corresponding state names. We create from `US_states` two dataframes: `US_states_user` and `US_states_beer`. 

We add the corresponding postal abbreviation to the user and brewery dataframes. We do this by merging the `df_BA` and `df_RB` dataframes with `US_states_user` and `US_states_beer` respectively. 

We then finish processing the locations by dropping the state name in the location column.


In [ ]:
# Creating the US_states dataframe
US_states = pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-2:US')[0] 
US_states['Subdivision name (en)'] = US_states['Subdivision name (en)'].apply(lambda x: 'United States, ' + x) 
US_states['Code'] = US_states['Code'].apply(lambda x: x[3:]) 
US_states.drop(columns=['Subdivision category'], inplace=True) 

# Creating the two dataframes from the US_states dataframe
US_states_user=US_states.rename(columns={'Subdivision name (en)':'location_user', 'Code':'US_Code_User'}) 
US_states_beer=US_states.rename(columns={'Subdivision name (en)':'beers_location', 'Code':'US_Code_Beer'}) 

# Merging to add the postal abbreviations to the RateBeer and BeerAdvocate dataframes
df_BA=pd.merge(US_states_beer, df_BA, how='outer', on='beers_location') 
df_BA=pd.merge(US_states_user, df_BA, how='outer', on='location_user') 


df_RB=pd.merge(US_states_beer, df_RB, how='outer', on='beers_location')
df_RB=pd.merge(US_states_user, df_RB, how='outer', on='location_user')

def keep_United_States_if_in_the_string(x): 
    if 'United States' in x: 
        return 'United States' 
    else: 
        return x 

# for the location and nationalities we kept only 'United States' and removed the State name after the comma for ploting.

df_BA['beers_location']=df_BA['beers_location'].apply(lambda x: str(x)) 
df_BA['beers_location']=df_BA['beers_location'].apply(lambda x: keep_United_States_if_in_the_string(x)) 
df_BA['location_user']=df_BA['location_user'].apply(lambda x: str(x)) 
df_BA['location_user']=df_BA['location_user'].apply(lambda x: keep_United_States_if_in_the_string(x)) 

df_RB['beers_location']=df_RB['beers_location'].apply(lambda x: str(x))
df_RB['beers_location']=df_RB['beers_location'].apply(lambda x: keep_United_States_if_in_the_string(x))
df_RB['location_user']=df_RB['location_user'].apply(lambda x: str(x))
df_RB['location_user']=df_RB['location_user'].apply(lambda x: keep_United_States_if_in_the_string(x))


### Exploring where users come from

In order to better define our analysis, we will first explore the geographical distribution of the users. To do so, we will first create a dataframe containing the number of users per country. We will then plot the distribution of the users on a world map. We do this for both dataframes `df_BA` and `df_RB`.

In [ ]:
count_country_users_RB=df_RB.groupby('location_user').count()['user_name'].to_frame().reset_index()
count_country_users_RB=count_country_users_RB.rename(columns={'user_name':'count_users'})

fig = px.choropleth(count_country_users_RB, 
                    locations='location_user',  
                    locationmode='country names',  
                    scope="world", 
                    color='count_users', 
                    )
fig.update_layout(title_text='RateBeer - Number of users per country') 
fig.show()

In [ ]:
count_country_users_BA=df_BA.groupby('location_user').count()['user_name'].to_frame().reset_index()
count_country_users_BA=count_country_users_BA.rename(columns={'user_name':'count_users'})

fig = px.choropleth(count_country_users_BA, 
                    locations='location_user',  
                    locationmode='country names',  
                    scope="world", 
                    color='count_users',  
                    )
fig.update_layout(title_text='BeerAdvocate - Number of users per country') 
fig.show()

### Focus on the US

We want to focus on the US for our analysis. We therefore create two new dataframes `BA_US` and `RB_US` that contain only the reviews of the US based users.

In [ ]:
RB_US = df_RB[df_RB['location_user'] == 'United States']
BA_US = df_BA[df_BA['location_user'] == 'United States']

Let's quickly check if some states have very few reviews.

In [ ]:
RB_US['US_Code_User'].value_counts()[RB_US['US_Code_User'].value_counts() < 1000]

In [ ]:
BA_US['US_Code_User'].value_counts()[BA_US['US_Code_User'].value_counts() < 1000]

As we can see the US territories have 1 review each. We will therefore remove them from the dataframe since they will not be useful for our analysis.

In [ ]:
RB_US = RB_US[~RB_US['US_Code_User'].isin(['DC', 'AS', 'GU', 'MP', 'PR', 'VI', 'UM'])]
BA_US = BA_US[~BA_US['US_Code_User'].isin(['DC', 'AS', 'GU', 'MP', 'PR', 'VI', 'UM'])]

Let's now explore the distribution of the users in the US. We will plot the distribution of the users on a map of the US.

In [ ]:
count_state_users_RB=RB_US.groupby('US_Code_User').count()['user_name'].to_frame().reset_index()
count_state_users_RB=count_state_users_RB.rename(columns={'user_name':'count_users'})

fig = px.choropleth(count_state_users_RB, 
                    locations='US_Code_User',  
                    locationmode='USA-states',  
                    scope="usa", 
                    color='count_users',  
                    )
fig.update_layout(title_text='RateBeer - Number of users per country') 
fig.show()

In [ ]:
count_state_users_BA=BA_US.groupby('US_Code_User').count()['user_name'].to_frame().reset_index()
count_state_users_BA=count_state_users_BA.rename(columns={'user_name':'count_users'})

fig = px.choropleth(count_state_users_BA, 
                    locations='US_Code_User',  
                    locationmode='USA-states',  
                    scope="usa", 
                    color='count_users',  
                    )
fig.update_layout(title_text='BeerAdvocate - Number of users per country') 
fig.show()

### Dealing with missing values

We start by checking the number of missing values in each column of the dataframes. 

In [ ]:
print(RB_US.isnull().sum())
print("Percentage of NaN values in RB_US: ", (RB_US['text'].isnull().sum()/len(RB_US))*100, "%")

In [ ]:
print(BA_US.isnull().sum())
print("Percentage of NaN values in BA_US: ", (BA_US['appearance'].isnull().sum()/len(BA_US))*100, "%")

After our processing, we can see that for `RB_US` we have no NaN values for the rating columns and NaN values for the text column. For `BA_US` we have no NaN values for the text column and NaN values for the rating columns.
If we look at how much these NaN values represent in the dataframes, we can see that for `RB_US` the NaN values represent 0.005% of the data and for `BA_US` the NaN values represent 0.6% of the data. Thus we can drop these rows without losing too much information.

In [ ]:
RB_US = RB_US.dropna(subset=['text'], how='all')
BA_US = BA_US.dropna(subset=['appearance','aroma','palate','taste','overall'], how='all')

### Checking data types

In [ ]:
RB_US.dtypes

In [ ]:
BA_US.dtypes

We first have to convert the `date` column to a datetime object. We do this for both `RB_US` and `BA_US`.


In [ ]:
RB_US['date'] = RB_US['date'].apply(datetime.datetime.fromtimestamp)
BA_US['date'] = BA_US['date'].apply(datetime.datetime.fromtimestamp)

We finally have our two dataframes `RB_US` and `BA_US` that we will use for the analysis.

## Merging the dataframes

We want to have as much data as possible for our analysis. We therefore need to merge ratings data from both BeerAdvocate and RateBeer. We need to normalize the data because the distributions are different between the two sites. We follow the same procedure as in Lederrey-West paper ([Lederrey-West_WWW-18](https://dlab.epfl.ch/people/west/pub/Lederrey-West_WWW-18.pdf)):
We observe that the mean of the ratings is higher for BeerAdvocate than for Ratebeer. Moreover, when we observe the mean and std of rating over the course of time, *the mean increases, while the standard deviation decreases, from year to year. Assuming that the inherent quality of beers being rated stays roughly constant, the rising mean may be interpreted as score inflation, while the sinking standard deviation could indicate a consolidating consensus about what should constitute the score of an average beer.* (Lederrey-West_WWW-18)

Thus we perform a z-score normalization of the ratings : *for each site and each year, we compute the mean and standard deviation over all ratings.We then subtract the mean of year t from all ratings submitted in year t and divide them by the standard deviation of year t , such that each year’s set of ratings has mean 0 and standard deviation 1.* (Lederrey-West_WWW-18)

Finally we simply merge the two dataframes into one containings all the raitings from both BeerAdvocate and RateBeer with normalized scores for each feature.

In [ ]:
def normalize_data(df):
    """
    Normalize the data : compute z scores for each feature (look, smell/aroma, taste, feel/palate, overall & rating)
    we do it for each diffrent year to take the temporal drift of the mean and variance into account
    """
    df['year'] = df['date'].apply(lambda x: x.year)
    for year in df['year'].unique():
        df_year_ = df[df['year'] == year]
        df_year = df_year_.copy()
        for feature in ['appearance','aroma','taste','palate','overall','rating']:
            df_year[feature] = (df_year[feature] - df_year[feature].mean())/df_year[feature].std()
        df.loc[df['year'] == year] = df_year
    return df

In [ ]:
def normalize_data(df):
    """
    Normalize the data : compute z scores for each feature (look, smell/aroma, taste, feel/palate, overall & rating)
    we do it for each diffrent year to take the temporal drift of the mean and variance into account
    """
    df['year'] = df['date'].apply(lambda x: x.year)
    for year in df['year'].unique():
        df_year_ = df[df['year'] == year]
        df_year = df_year_.copy()
        for feature in ['appearance','aroma','taste','palate','overall','rating']:
            df_year[feature] = scale(df_year[feature])
        df.loc[df['year'] == year] = df_year
    return df

In [ ]:
# normalize the data
RB_US_norm = normalize_data(RB_US)
BA_US_norm = normalize_data(BA_US)

# merge the two dataframes
df_ratings = pd.concat([RB_US_norm, BA_US_norm], ignore_index=True)
df_ratings

We can observe the distribution of the scores for the different features for normalized BeerAdvocate (blue), RateBeer (red) and merged (white) ratings.

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(15, 7))
sns.histplot(RB_US_norm['appearance'], bins=np.arange(-5, 3, 0.25), ax=ax[0,0], color='red')
sns.histplot(BA_US_norm['appearance'], bins=np.arange(-5, 3, 0.25), ax=ax[0,0], color='blue')
sns.histplot(df_ratings['appearance'], bins=np.arange(-5, 3, 0.25), ax=ax[0,0], color='white')
sns.histplot(RB_US_norm['aroma'], bins=np.arange(-5, 3, 0.25), ax=ax[0,1], color='red')
sns.histplot(BA_US_norm['aroma'], bins=np.arange(-5, 3, 0.25), ax=ax[0,1], color='blue')
sns.histplot(df_ratings['aroma'], bins=np.arange(-5, 3, 0.25), ax=ax[0,1], color='white')
sns.histplot(RB_US_norm['taste'], bins=np.arange(-5, 3, 0.25), ax=ax[0,2], color='red')
sns.histplot(BA_US_norm['taste'], bins=np.arange(-5, 3, 0.25), ax=ax[0,2], color='blue')
sns.histplot(df_ratings['taste'], bins=np.arange(-5, 3, 0.25), ax=ax[0,2], color='white')
sns.histplot(RB_US_norm['palate'], bins=np.arange(-5, 3, 0.25), ax=ax[1,0], color='red')
sns.histplot(BA_US_norm['palate'], bins=np.arange(-5, 3, 0.25), ax=ax[1,0], color='blue')
sns.histplot(df_ratings['palate'], bins=np.arange(-5, 3, 0.25), ax=ax[1,0], color='white')
sns.histplot(RB_US_norm['overall'], bins=np.arange(-5, 3, 0.25), ax=ax[1,1], color='red')
sns.histplot(BA_US_norm['overall'], bins=np.arange(-5, 3, 0.25), ax=ax[1,1], color='blue')
sns.histplot(df_ratings['overall'], bins=np.arange(-5, 3, 0.25), ax=ax[1,1], color='white')
sns.histplot(RB_US_norm['rating'], bins=np.arange(-5, 3, 0.25), ax=ax[1,2], color='red')
sns.histplot(BA_US_norm['rating'], bins=np.arange(-5, 3, 0.25), ax=ax[1,2], color='blue')
sns.histplot(df_ratings['rating'], bins=np.arange(-5, 3, 0.25), ax=ax[1,2], color='white')
plt.show()

# Analysis

## 1 - Quick statistics from the data

### 1.1 - Number of reviews per brewery
First we count the number of reviews made about each brewery. We select only the top10 to display them on a barplot.

In [ ]:
#sort the breweries with the most reviews
brewery_counts = df_ratings['brewery_name'].value_counts().sort_values(ascending=False).head(10)

fig = px.bar(brewery_counts, x = brewery_counts.index, y = brewery_counts.values, title = 'Top 10 Breweries with the most reviews',color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_layout(
    xaxis = dict(title = "Breweries"),
    yaxis = dict(title = "Number of reviews")
)
fig.show()
#comment the code below if you don't want to generate the html
#fig.write_html('breweries_reviews.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

### 1.2 - Number of reviews per beer
Secondly, we count the number of reviews made about each beer. We select only the top10 to display them on a barplot.

In [ ]:
#sort the beers with the most reviews
beers_counts = df_ratings['beer_name'].value_counts().sort_values(ascending=False).head(10)

fig = px.bar(beers_counts, x = beers_counts.index, y = beers_counts.values, title = "Reviews by Beer",color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_layout(
    xaxis = dict(title = "Beer name"),
    yaxis = dict(title = "Number of reviews")
)
fig.show()
#comment the code below if you don't want to generate the html
#fig.write_html('beer_reviews_BA.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

### 1.3 - Number of reviews per state
Finaly comes the tricky part, what do we mean by __state__ ?  
We count the number of reviews made for each user location (state) using ``US_Code_User`` and we select only the top10 to display them on a barplot.
Then count the number of reviews made for each beer location (state) using ``US_Code_Beer`` and we select only the top10 to display them on a barplot.

In [ ]:
#sort the user locations with the most reviews
states_counts = df_ratings['US_Code_User'].value_counts().sort_values(ascending=False).head(10)

fig = px.bar(states_counts, x = states_counts.index, y = states_counts.values, title = "Beer Reviews by State by User",color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_traces(marker_color='#DC3220')
fig.update_layout(
    xaxis = dict(title = "States"),
    yaxis = dict(title = "Number of reviews by User")
)
fig.show()
#comment the code below if you don't want to generate the html
#fig.write_html('states_reviews_users_BA.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

In [ ]:
#sort the beer locations with the most reviews
states_counts = df_ratings['US_Code_Beer'].value_counts().sort_values(ascending=False).head(10)

fig = px.bar(states_counts, x = states_counts.index, y = states_counts.values, title = 'Beer Reviews by State',color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_layout(
    xaxis = dict(title = "States"),
    yaxis = dict(title = "Number of reviews")
)
fig.show()
#comment the code below if you don't want to generate the html
#fig.write_html('states_reviews_beers_BA.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

## 2 - Digging deeper, fine-tuning our analysis
### 2.1 - PCA on the rating aspects

First we load the necessary dataframes for the analysis. We use the `RB_US` and `BA_US` dataframes that we created in the preprocessing part. We will merge them into one dataframe `data_4_PCA` that we will use for the PCA analysis. We also scale the dataframe to have a mean of 0 and a standard deviation of 1 which is necessary for the PCA analysis.

In [ ]:
features = ['appearance','aroma','taste','palate']
target = 'overall'

In [ ]:
data_4_PCA = df_ratings
features_PCA = data_4_PCA[features]
target_PCA = data_4_PCA[target]

We then start our PCA analysis.

In [ ]:
pca = PCA(n_components=2)
pca.fit(features_PCA)
explained_variance = pca.explained_variance_ratio_
explained_variance

The first component explains 73.2 % of the variance and the second 12.1 % of the variance. 

In [ ]:
scores = pca.transform(features_PCA)
scores_df = pd.DataFrame(scores, columns=['PC1', 'PC2'])
print(scores_df)
loadings = pca.components_.T
df_loadings = pd.DataFrame(loadings, columns=['PC1', 'PC2'], index=features)
df_loadings.abs()

As we can see for the first component, which explains 73.2 % of the variance, the most important feature is taste. 

### 2.2 - Multiple regression analysis on the rating aspects

To perform the multiple regression analysis, we will use the scikit-learn library. We will use the data we prepared in 2.1 : `features_PCA` and `target_PCA`. We use a the LinearRegression model from scikit-learn to fit the model on the data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_PCA, target_PCA, test_size=0.2, random_state=42)
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

We also also want to determine the coefficient of determination $R^2$ of the prediction which will tell us if our model is good or not. 

In [ ]:
r2 = regressor.score(X_test, y_test)
print(f"Coefficient of determination: {r2:.2f}")

We can see that the coefficient of determination is 0.75 which is a good value. The data was centered in 2.1 and LinearRegression automatically adds an intercept. Thus the coefficient of determination R^2 is the centered R^2.

In [ ]:
weights = regressor.coef_
print(weights)

Finally as expected we see that the most important feature is taste. The next most important features are palate and aroma, it seems palate is a bit more important than aroma.

### Correlation between the rating aspects

We will explore the correlation between the rating aspects by simply plotting a heatmap of the correlation matrix.


In [ ]:
multi_reg_df = pd.concat([pd.DataFrame(features_PCA), pd.DataFrame(target_PCA)], axis=1)
multi_reg_df.columns = [features + ['overall']]

In [ ]:
# Create a correlation matrix 
corr_metrics = multi_reg_df.corr()
corr_metrics.style.background_gradient(cmap='Blues')

### Let's dig deeper 

We take a look at 2 beers with low and high scores for taste. We will look at the distribution of the scores for the different features for these two beers.

#### Mild taste beers

We choose two known beer types for their mild taste : American Adjunct Lager (Budweiser) and Pale Lager. 

In [ ]:
AALager = df_ratings[features + [target]][df_ratings['style'] =='American Adjunct Lager']
AALager_mean = AALager[features + [target]].mean().to_frame()
AALager_mean.columns = ['mean']
print(AALager_mean)

corr_metrics = AALager.corr()
corr_metrics.style.background_gradient(cmap='Blues')

In [ ]:
Pale_lager = df_ratings[features + [target]][df_ratings['style'] =='Pale Lager']
Pale_lager_mean = Pale_lager[features + [target]].mean().to_frame()
Pale_lager_mean.columns = ['mean']
print(Pale_lager_mean)

corr_metrics = Pale_lager.corr()
corr_metrics.style.background_gradient(cmap='Blues')

#### Strong taste beers

We choose two known beer types for their strong taste : IPA and Stout.

In [ ]:
ipa = df_ratings[features + [target]][df_ratings['style'] =='India Pale Ale (IPA)']
ipa_mean = ipa[features + [target]].mean().to_frame()
ipa_mean.columns = ['mean']
print(ipa_mean)

corr_metrics = ipa.corr()
corr_metrics.style.background_gradient(cmap='Blues')

In [ ]:
stout = df_ratings[features + [target]][df_ratings['style'] == 'Stout'] 
stout[features + [target]].mean().to_frame()

stout = df_ratings[features + [target]][df_ratings['style'] =='Stout']
stout_mean = stout[features + [target]].mean().to_frame()
stout_mean.columns = ['mean']
print(stout_mean)

corr_metrics = stout.corr()
corr_metrics.style.background_gradient(cmap='Blues')

In [ ]:
beerstyle_taste = df_ratings[df_ratings['style'].isin(['India Pale Ale (IPA)', 'Stout', 'Pale Lager', 'American Adjunct Lager'])].groupby(['style', 'US_Code_User']).size().reset_index(name='count_users')
beerstyle_taste.index = beerstyle_taste['US_Code_User']
beerstyle_taste = beerstyle_taste.pivot(columns='style', values='count_users')
beerstyle_taste['total'] = beerstyle_taste.sum(axis=1)
beerstyle_taste.reset_index(inplace=True)
fig = px.choropleth(beerstyle_taste, 
                    locations='US_Code_User',  
                    locationmode='USA-states',  
                    scope="usa", 
                    color='total',
                    hover_data=['India Pale Ale (IPA)', 'Stout', 'Pale Lager', 'American Adjunct Lager'],
                    )
fig.update_layout(title_text='RateBeer - Number of users per country') 
fig.show()

In [ ]:
beerstyle_taste

In [ ]:
fig = px.choropleth(
    beerstyle_taste,
    locationmode='USA-states',  
    scope="usa",
    locations="US_Code_User", 
    color="total", 
)

# Add a dropdown menu
updatemenus = list([
    dict(
        buttons=list([
            dict(
                label="Total",
                args=[{"color": "total"}, {"title": "total"}],
                method="update",
            ),
            dict(
                label="American Adjunct Lager",
                args=[{"color": "American Adjunct Lager"}, {"title": "American Adjunct Lager"}],
                method="update",
            ),
            dict(
                label="Pale Lager",
                args=[{"color": "Pale Lager"}, {"title": "Pale Lager"}],
                method="update",
            ),
            dict(
                label="India Pale Ale (IPA)",
                args=[{"color": "India Pale Ale (IPA)"}, {"title": "India Pale Ale (IPA)"}],
                method="update",
            ),
            dict(
                label="Stout",
                args=[{"color": "Stout"}, {"title": "Stout"}],
                method="update",
            ),
        ]),
        direction = 'down',
        pad = {'r': 10, 't': 10},
        showactive = False,
        x = 0.1,
        xanchor = 'left',
        y = 1.1,
        yanchor = 'top' 
    ),
])

plt.show()


## 3 - Sentiment analysis

First step is to run the analysis over the whole dataframe ``df_ratings``, as it is a memory intensive task, we will chunk the dataframe into smaller dataframes of 200000 rows each. We will then run the analysis on each chunk and store the results in a new file ``df_sentiment.csv``. We will then load the results from this file and continue the analysis.  
Using the TextBlob library we compute 2 scores : sentiment and objectivity.

In [ ]:
chunk_size = 200000
start_row = 0
end_row = start_row + chunk_size
df_sentiment = df_ratings

while start_row < len(df_sentiment):
  df_chunk = df_sentiment.iloc[start_row:end_row]
  df_chunk['sentiment'] = df_chunk['text'].apply(lambda x: TextBlob(str(x)).sentiment[0])
  df_chunk['objectivity'] = df_chunk['text'].apply(lambda x: TextBlob(str(x)).sentiment[1])
  #Write the chunk of data into a file
  df_chunk.to_csv('/content/drive/MyDrive/ada-2022-projet-datalcoholic/df_sentiment.csv', index=False, mode='a')
  df_chunk.drop(df_chunk.index, inplace=True)
  #Update counters
  start_row += chunk_size
  end_row += chunk_size

Let's do some processing to get the best out of this dataframe

In [ ]:
df_sentiment = df_sentiment[df_sentiment['sentiment'] != 'sentiment'] #error when chucking the data
df_sentiment.replace([np.inf, -np.inf], np.nan).dropna()
df_sentiment.dropna()
df_sentiment['sentiment'] = df_sentiment['sentiment'].apply(lambda x : float(x))
df_sentiment['rating'] = df_sentiment['rating'].apply(lambda x : float(x))
df_sentiment['year'] = df_sentiment['year'].apply(lambda x : float(x)) # convert year to float for future use

In [ ]:
df_sentiment.to_csv('../Data/df_sentiment.csv')

In [ ]:
df_sentiment = pd.read_csv('../Data/df_sentiment.csv')

### 3.1 - Sentiment analysis and Rating distributions

As users are asked to grade the beer and give a textual review, let's investigate if these two are correlated. We will first do a simple comparison through a boxplot of the sentiment score and the rating score for the top10 beers having the most reviews.

In [ ]:
df_top10 = df_sentiment['beer_name'].value_counts().sort_values(ascending=False).head(10).reset_index()
df_top10_stats = df_sentiment[df_sentiment['beer_name'].isin(df_top10['beer_name'])].reset_index()

fig = go.Figure()
fig.add_trace(go.Box(y=df_top10_stats['sentiment'], x=df_top10_stats['beer_name'], name='Sentiment score', boxpoints=False, marker_color='#10128F'))
fig.add_trace(go.Box(y=df_top10_stats['rating'], x=df_top10_stats['beer_name'], name='Rating score', boxpoints=False, marker_color='#E83101'))

fig.update_layout(
    title="Sentiment vs Rating",
    xaxis_title="Top 10 Most reviewed Beers",
    yaxis_title="Sentiment Score and rating distributions",
    boxmode='group' # group together boxes of the different traces for each value of x
)
fig.show()
#comment the code below if you don't want to generate the html
#fig.write_html('boxplot_sentiment_rating.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

Visualizing the distribution of the sentiment score next to the distribution of the rating score do not really give us informations about a possible correlation between the two. One observation we can made is that the rating scores are way more spread around the median than the sentiment scores. It can be interesting to make a regression between these two variables to see if there is a linear correlation.

### 3.2 - Linear regression

We will use the ``scikit-learn`` library to perform the linear regression and the ``LinearRegression`` model to fit the model on the data.  
We chose to do it for 3 of the top 10 beers as well as 3 random beers having at least 1000 reviews.

In [ ]:
#--------- Bud Light ---------#
df_BL = df_sentiment[df_sentiment['beer_name'] == 'Bud Light']

scaler = StandardScaler()
regression_BL = df_BL[['rating', 'sentiment']]
scaled_data = scaler.fit_transform(regression_BL)

df_scaled_BL = pd.DataFrame(scaled_data, columns=regression_BL.columns)
x = np.array(df_scaled_BL['rating']).reshape(-1, 1)
y = np.array(df_scaled_BL['sentiment'])

model = linear_model.LinearRegression()
model.fit(x, y)
predicted_BL = model.predict(x)
r2_BL = r2_score(df_scaled_BL['sentiment'], predicted_BL) #r2 score


#--------- Newcastle Brown Ale ---------#
df_NBA = df_sentiment[df_sentiment['beer_name'] == 'Newcastle Brown Ale']

scaler = StandardScaler()
regression_NBA = df_NBA[['rating', 'sentiment']]
scaled_data = scaler.fit_transform(regression_NBA)

df_scaled_NBA = pd.DataFrame(scaled_data, columns=regression_NBA.columns)
x = np.array(df_scaled_NBA['rating']).reshape(-1, 1)
y = np.array(df_scaled_NBA['sentiment'])

model = linear_model.LinearRegression()
model.fit(x, y)
predicted_NBA = model.predict(x)
r2_NBA = r2_score(df_scaled_NBA['sentiment'], predicted_NBA) #r2 score


#--------- Brooklyn Black Chocolate Stout ---------#
df_BBC = df_sentiment[df_sentiment['beer_name'] == 'Brooklyn Black Chocolate Stout']

scaler = StandardScaler()
regression_BBC = df_BBC[['rating', 'sentiment']]
scaled_data = scaler.fit_transform(regression_BBC)

df_scaled_BBC = pd.DataFrame(scaled_data, columns=regression_BBC.columns)
x = np.array(df_scaled_BBC['rating']).reshape(-1, 1)
y = np.array(df_scaled_BBC['sentiment'])

model = linear_model.LinearRegression()
model.fit(x, y)
predicted_BBC = model.predict(x)
r2_BBC = r2_score(df_scaled_BBC['sentiment'], predicted_BBC) #r2 score


#--------- Tuppers Hop Pocket Ale ---------#
df_THPA = df_sentiment[df_sentiment['beer_name'] == 'Tuppers Hop Pocket Ale']

scaler = StandardScaler()
regression_THPA = df_THPA[['rating', 'sentiment']]
scaled_data = scaler.fit_transform(regression_THPA)

df_scaled_THPA = pd.DataFrame(scaled_data, columns=regression_THPA.columns)
x = np.array(df_scaled_THPA['rating']).reshape(-1, 1)
y = np.array(df_scaled_THPA['sentiment'])

model = linear_model.LinearRegression()
model.fit(x, y)
predicted_THPA = model.predict(x)
r2_THPA = r2_score(df_scaled_THPA['sentiment'], predicted_THPA) #r2 score


#--------- Great Lakes Oktoberfest ---------#
df_GLO = df_sentiment[df_sentiment['beer_name'] == 'Great Lakes Oktoberfest']

scaler = StandardScaler()
regression_GLO = df_GLO[['rating', 'sentiment']]
scaled_data = scaler.fit_transform(regression_GLO)

df_scaled_GLO = pd.DataFrame(scaled_data, columns=regression_GLO.columns)
x = np.array(df_scaled_GLO['rating']).reshape(-1, 1)
y = np.array(df_scaled_GLO['sentiment'])

model = linear_model.LinearRegression()
model.fit(x, y)
predicted_GLO = model.predict(x)
r2_GLO = r2_score(df_scaled_GLO['sentiment'], predicted_GLO) #r2 score


#--------- St-Feuillien Triple ---------#
df_SFT = df_sentiment[df_sentiment['beer_name'] == 'St-Feuillien Triple']

scaler = StandardScaler()
regression_SFT = df_SFT[['rating', 'sentiment']]
scaled_data = scaler.fit_transform(regression_SFT)

df_scaled_SFT = pd.DataFrame(scaled_data, columns=regression_SFT.columns)
x = np.array(df_scaled_SFT['rating']).reshape(-1, 1)
y = np.array(df_scaled_SFT['sentiment'])

model = linear_model.LinearRegression()
model.fit(x, y)
predicted_SFT = model.predict(x)
r2_SFT = r2_score(df_scaled_SFT['sentiment'], predicted_SFT) #r2 score

We generate 2 plots having each 3 subplots in one column to facilitate the comparison.

In [ ]:
fig = make_subplots(
    rows=3, cols=1, 
    subplot_titles=("Bud Light", 'Newcastle Brown Ale', 'Brooklyn Black Chocolate Stout'),
    shared_xaxes=True,
    x_title='Rating',
    y_title='Sentiment Score',
    vertical_spacing = 0.04)

#BL
fig.append_trace(go.Scatter(
    x=df_scaled_BL['rating'], 
    y=df_scaled_BL['sentiment'], 
    mode='markers', marker_color = '#8B9EF8'),
    row=1, col=1)

fig.append_trace(go.Scatter(
    x=df_scaled_BL['rating'], 
    y=predicted_BL, 
    mode='lines', marker_color = '#F31E18'), 
    row=1, col=1)

#NBA
fig.append_trace(go.Scatter(
    x=df_scaled_NBA['rating'], 
    y=df_scaled_NBA['sentiment'], 
    mode='markers', marker_color = '#8B9EF8'),
    row=2, col=1)

fig.append_trace(go.Scatter(
    x=df_scaled_NBA['rating'], 
    y=predicted_NBA, 
    mode='lines', marker_color = '#F31E18'), 
    row=2, col=1)

#BBC
fig.append_trace(go.Scatter(
    x=df_scaled_BBC['rating'], 
    y=df_scaled_BBC['sentiment'], 
    mode='markers', marker_color = '#8B9EF8'),
    row=3, col=1)

fig.append_trace(go.Scatter(
    x=df_scaled_BBC['rating'], 
    y=predicted_BBC, 
    mode='lines', marker_color = '#F31E18'), 
    row=3, col=1)

fig.update_traces(marker={'size': 3}, showlegend=False)
fig.update_xaxes(range=[-4,4])
fig.update_yaxes(range=[-5,5])
fig.update_layout(height=650, width=500)
fig.add_annotation(x=-3,y=4,xref='x1',yref='y1',
                   text="R² =" + str(round(r2_BL,3-int(math.floor(math.log10(abs(r2_BL))))-1)),
                   showarrow=False, row=1, col=1)
fig.add_annotation(x=-3,y=4,xref='x2',yref='y2',
                   text="R² =" + str(round(r2_NBA,3-int(math.floor(math.log10(abs(r2_NBA))))-1)),
                   showarrow=False, row=2, col=1)
fig.add_annotation(x=-3,y=4,xref='x3',yref='y3',
                   text="R² =" + str(round(r2_BBC,3-int(math.floor(math.log10(abs(r2_BBC))))-1)),
                   showarrow=False, row=3, col=1)
fig.show()
#comment the code below if you don't want to generate the html
#fig.write_html('regression_1.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

In [ ]:
fig = make_subplots(
    rows=3, cols=1, 
    subplot_titles=("uppers Hop Pocket Ale", 'Great Lakes Oktoberfest', 'St-Feuillien Triple'),
    shared_xaxes=True,
    x_title='Rating',
    y_title='Sentiment Score',
    vertical_spacing = 0.04)

#THPA
fig.append_trace(go.Scatter(
    x=df_scaled_THPA['rating'], 
    y=df_scaled_THPA['sentiment'], 
    mode='markers', marker_color = '#8B9EF8'),
    row=1, col=1)

fig.append_trace(go.Scatter(
    x=df_scaled_THPA['rating'], 
    y=predicted_THPA, 
    mode='lines', marker_color = '#F31E18'), 
    row=1, col=1)

#GLO
fig.append_trace(go.Scatter(
    x=df_scaled_GLO['rating'], 
    y=df_scaled_GLO['sentiment'], 
    mode='markers', marker_color = '#8B9EF8'),
    row=2, col=1)

fig.append_trace(go.Scatter(
    x=df_scaled_GLO['rating'], 
    y=predicted_GLO, 
    mode='lines', marker_color = '#F31E18'), 
    row=2, col=1)

#SFT
fig.append_trace(go.Scatter(
    x=df_scaled_SFT['rating'], 
    y=df_scaled_SFT['sentiment'], 
    mode='markers', marker_color = '#8B9EF8'),
    row=3, col=1)

fig.append_trace(go.Scatter(
    x=df_scaled_SFT['rating'], 
    y=predicted_SFT, 
    mode='lines', marker_color = '#F31E18'), 
    row=3, col=1)

fig.update_traces(marker={'size': 3}, showlegend=False)
fig.update_xaxes(range=[-4,4])
fig.update_yaxes(range=[-5,5])
fig.update_layout(height=650, width=500)
fig.add_annotation(x=-3,y=4,xref='x1',yref='y1',
                   text="R² =" + str(round(r2_THPA,3-int(math.floor(math.log10(abs(r2_THPA))))-1)),
                   showarrow=False, row=1, col=1)
fig.add_annotation(x=-3,y=4,xref='x2',yref='y2',
                   text="R² =" + str(round(r2_GLO,3-int(math.floor(math.log10(abs(r2_GLO))))-1)),
                   showarrow=False, row=2, col=1)
fig.add_annotation(x=-3,y=4,xref='x3',yref='y3',
                   text="R² =" + str(round(r2_SFT,3-int(math.floor(math.log10(abs(r2_SFT))))-1)),
                   showarrow=False, row=3, col=1)
fig.show()
#comment the code below if you don't want to generate the html
#fig.write_html('regression_2.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

## Time series analysis

In [ ]:
df_ratings_notnormalized = pd.concat([RB_US, BA_US], ignore_index=True)
#create dataframe with mean overall rating over time using a monthly frequency
df_ratings['date'] = pd.to_datetime(df_ratings_notnormalized['date'])
df_ratings = df_ratings.set_index('date')
df_ratings = df_ratings.resample('M').mean()
df_ratings = df_ratings.reset_index()
df_ratings = df_ratings.dropna()

In [ ]:
#plot the mean overall rating over time
fig, ax = plt.subplots(figsize=(15, 7))
ax.plot(df_ratings['date'], df_ratings['overall'], color='blue')
ax.set_xlabel('Date')
ax.set_ylabel('Mean Overall Rating')
ax.set_title('Mean Overall Rating Over Time')
plt.show()

In [ ]:
#open csv file called 'results_by_state_V2.csv' and read it into a dataframe
df_state = pd.read_csv('results_by_state_V2.csv')


In [ ]:
df_state

In [ ]:
WV_evolution= df_state[df_state['State'] == 'WV']

In [ ]:
df_ratings_notnormalized

In [ ]:
WV_evolution

In [ ]:
WV_reviews = df_ratings_notnormalized[df_ratings_notnormalized['US_Code_User'] == 'WV']

In [ ]:
WV_reviews

In [ ]:
# plot number of reviews over time 
number_of_reviews_WV = WV_reviews.groupby('year').count()
number_of_reviews_WV 